## 1. セットアップ

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 自作モジュールの読み込み
sys.path.append(os.path.abspath('..'))
from configs.config import *
from src.runner import Runner
from src.model_resnet import ModelResNet50
from src.util import Logger, Util, Metric

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 2. ロガーの設定

In [ ]:
# ロガーの初期化
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    """run名の作成"""
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized")

## 3. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv'))
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv'))

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

# ✅ リークしないCV戦略: quarter_sessionでグループ化
# 理由: 同一session内のframeは時間的に連続 → 必ずsession単位でグループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")
print(f"\n  Top 5 groups by size:")
print(df_train['group'].value_counts().head())

# データの確認
print(f"\n【Data Overview】")
print(df_train.head())
print(f"\nLabel distribution:")
print(df_train['label_id'].value_counts().sort_index())

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

# グループ列作成（quarter_session）
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)
print(f"Groups: {df_train['group'].nunique()}")

## 4. パラメータ設定

In [ ]:
# メモ
memo = "ResNet50 Baseline - Prototype method"

# run_name
# run_name = get_run_name(model_type="resnet50_baseline")
run_name = 'resnet50_baseline_202512132023'  # 既存runを使う場合

print(f"Run name: {run_name}")
print(f"Memo: {memo}")

In [ ]:
# モデルパラメータ
params = {
    'model_name': 'resnet50',  # 'resnet50' or 'efficientnet_b0'
    'method': 'prototype',  # 'prototype' or 'knn'
    'k': 5,  # KNN法のk
    'threshold': 0.5,  # 類似度閾値（1st nearest）
    'min2_threshold': 0.3,  # 類似度閾値（2nd nearest）
    'batch_size': 32,
    'num_workers': 4,  # DataLoaderの並列数
    'use_cache': True,  # 特徴量キャッシュを使用するか
}

# ✅ CV設定（リークしない戦略）
cv_setting = {
    'method': 'group',  # 'group' (推奨) or 'stratified_group'
    'group_col': 'group',  # 上で作成したquarter_session列
    'n_splits': 5,
    # 注: GroupKFoldはshuffleパラメータなし（時系列順を保つ）
}

print("=== Parameters ===")
for key, val in params.items():
    print(f"  {key}: {val}")

print("\n=== CV Settings ===")
for key, val in cv_setting.items():
    print(f"  {key}: {val}")
    
print("\n⚠️  CV Strategy:")
print("  - GroupKFold: リーク防止最優先（推奨）")
print("  - 同一session内のframeは必ず同じfoldに")
print("  - 訓練と検証でgroupが重複しない")

## 5. Runnerの作成

In [ ]:
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger,
)

logger.info(f"Memo: {memo}")

## 6-A. CV学習（推奨: CVスコアで性能確認）

In [ ]:
# CV学習実行
scores = runner.train_cv()

print(f"\nCV Mean Score: {np.mean(scores):.6f} (+/- {np.std(scores):.6f})")

In [ ]:
# CV ensemble予測
submission_cv = runner.predict_cv()

# 提出ファイル保存
save_path = runner.save_submission(submission_cv, suffix="cv")
print(f"Submission saved: {save_path}")

## 6-B. 全データ学習（オプション: CV不要の場合）

In [ ]:
# 全データで学習
# model_all = runner.train_all()

# 予測
# submission_all = runner.predict_all()

# 提出ファイル保存
# save_path = runner.save_submission(submission_all, suffix="all")
# print(f"Submission saved: {save_path}")

## 7. 結果分析

In [ ]:
# 予測分布の確認
pred_counts = submission_cv['label_id'].value_counts().sort_index()

# 訓練データとの比較
train_counts = df_train['label_id'].value_counts().sort_index()

comparison_df = pd.DataFrame({
    'train_count': train_counts,
    'pred_count': pred_counts
}).fillna(0).astype(int)

comparison_df['ratio'] = comparison_df['pred_count'] / comparison_df['train_count']

print("=== Train vs Prediction ===")
display(comparison_df)

In [ ]:
# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# カウント比較
comparison_df[['train_count', 'pred_count']].plot(kind='bar', ax=axes[0])
axes[0].set_title('Train vs Prediction Counts')
axes[0].set_xlabel('Label ID')
axes[0].set_ylabel('Count')
axes[0].legend(['Train', 'Prediction'])
axes[0].grid(axis='y', alpha=0.3)

# 比率
comparison_df['ratio'].plot(kind='bar', ax=axes[1], color='green', alpha=0.7)
axes[1].set_title('Prediction/Train Ratio')
axes[1].set_xlabel('Label ID')
axes[1].set_ylabel('Ratio')
axes[1].axhline(y=1.0, color='r', linestyle='--', alpha=0.5)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Unknown予測の分析
unknown_count = (submission_cv['label_id'] == -1).sum()
unknown_rate = unknown_count / len(submission_cv)

print(f"\n=== Unknown Predictions ===")
print(f"Count: {unknown_count} / {len(submission_cv)}")
print(f"Rate: {unknown_rate*100:.2f}%")

## 8. 閾値チューニング（画像コンペの改良点）

特徴量キャッシュにより、類似度計算なしで閾値だけを変更して高速実験が可能

In [ ]:
# 異なる閾値で予測（類似度キャッシュを再利用 = 超高速）
from src.util import Metric

# モデル読み込み（既に学習済み）
model = runner.build_model(i_fold=0)
model.load_model()

# Validation データ準備
tr, va = runner.create_train_valid_dataset(i_fold=0)

# 一度予測実行（類似度計算）
va_pred = model.predict(va)

# 複数の閾値で高速実験
threshold_results = []

for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
    for min2_threshold in [0.2, 0.3, 0.4]:
        # 類似度キャッシュから一瞬で予測
        predictions = model.predict_with_custom_threshold(threshold, min2_threshold)
        score = Metric.macro_f1(va['label_id'].values, predictions)
        
        threshold_results.append({
            'threshold': threshold,
            'min2_threshold': min2_threshold,
            'f1_score': score,
            'unknown_rate': (predictions == -1).mean()
        })

# 結果確認
df_results = pd.DataFrame(threshold_results).sort_values('f1_score', ascending=False)
print("=== Threshold Tuning Results ===")
display(df_results.head(10))

In [ ]:
# ヒートマップで可視化
pivot_table = df_results.pivot_table(
    values='f1_score',
    index='threshold',
    columns='min2_threshold'
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.4f', cmap='YlGnBu')
plt.title('F1 Score by Threshold Combinations')
plt.xlabel('min2_threshold')
plt.ylabel('threshold')
plt.tight_layout()
plt.show()

# 最適パラメータ
best_params = df_results.iloc[0]
print(f"\n=== Best Parameters ===")
print(f"Threshold: {best_params['threshold']}")
print(f"Min2 Threshold: {best_params['min2_threshold']}")
print(f"F1 Score: {best_params['f1_score']:.6f}")
print(f"Unknown Rate: {best_params['unknown_rate']*100:.2f}%")

## 9. キャッシュ管理

In [ ]:
from src.feature_cache import FeatureCache

cache = FeatureCache()

# 利用可能なキャッシュ一覧
cache.list_caches()

In [ ]:
# キャッシュクリア（必要に応じて）
# cache.clear()  # 全キャッシュクリア
# cache.clear('resnet50_xxxx')  # 特定キャッシュのみクリア

## 8. 次のステップ

### パラメータチューニング
- `threshold`, `min2_threshold` を調整
- `method` を 'knn' に変更して精度向上
- `model_name` を 'efficientnet_b0' に変更

### 改善案
1. **Data Augmentation**: 学習時に画像の拡張
2. **ArcFace導入**: Metric Learningで特徴量の質向上
3. **Multi-view統合**: Top viewの情報を活用
4. **時系列情報**: frame連続性を考慮